In [1]:
import pandas as pd
import numpy as np 
import datetime as dt 

#Load airbnb_price.csv, prices 
prices = pd.read_csv("Data/airbnb_price.csv")

#load airbnb_room_type.xlsx, xls 
xls = pd.ExcelFile("Data/airbnb_room_type.xlsx")

#Parse the first sheet from xls, room_types 
room_types = xls.parse(0)

#Load airbnb_last_review.tsv, reviews 
reviews = pd.read_csv("Data/airbnb_last_review.tsv", sep = "\t")

#Print the first five rows each dataframe 
print(prices.head(), "\n", room_types.head(), "\n", reviews.head())

   listing_id        price                nbhood_full
0        2595  225 dollars         Manhattan, Midtown
1        3831   89 dollars     Brooklyn, Clinton Hill
2        5099  200 dollars     Manhattan, Murray Hill
3        5178   79 dollars  Manhattan, Hell's Kitchen
4        5238  150 dollars       Manhattan, Chinatown 
    listing_id                                description        room_type
0        2595                      Skylit Midtown Castle  Entire home/apt
1        3831            Cozy Entire Floor of Brownstone  Entire home/apt
2        5099  Large Cozy 1 BR Apartment In Midtown East  Entire home/apt
3        5178            Large Furnished Room Near B'way     private room
4        5238         Cute & Cozy Lower East Side 1 bdrm  Entire home/apt 
    listing_id    host_name   last_review
0        2595     Jennifer   May 21 2019
1        3831  LisaRoxanne  July 05 2019
2        5099        Chris  June 22 2019
3        5178     Shunichi  June 24 2019
4        5238          

In [2]:
#Cleaning the price column 
#Remove whitespace and string characters
prices["price"]= prices["price"].str.replace("dollars","")

#Convert to numeric 
prices["price"]=pd.to_numeric(prices["price"])

#print descriptive statistic for the price column 
print(prices["price"].describe())

count    25209.000000
mean       141.777936
std        147.349137
min          0.000000
25%         69.000000
50%        105.000000
75%        175.000000
max       7500.000000
Name: price, dtype: float64


In [3]:
#Calculate average price 
#subset free_listing prices = $0
free_listings = prices["price"]==0 

#update prices by removing all free listings from prices 
prices = prices.loc[~free_listings]

#Calculate the average price, avg_price 
avg_price = round(prices["price"].mean(),2)

print("Average price per night for Airbnb listing in NYC is ${}.".format(avg_price))

Average price per night for Airbnb listing in NYC is $141.82.


In [4]:
#Comparing costs to the private rental market 

#Add column to prices Dataframe, price_per_month 
prices["price_per_month"] = prices["price"]*365/12 

#Calculate average_price per month 
average_price_per_month = round(prices["price_per_month"].mean(),2)

#Compare Airbnb and rental market 
print("Airbnb monthly cost are ${}, and Private Market monthly cost are {}.".format(average_price_per_month,"$3,100.00"))

Airbnb monthly cost are $4313.61, and Private Market monthly cost are $3,100.00.


In [5]:
#Cleaning room type column

#lowercase 
room_types["room_type"] = room_types["room_type"].str.lower()

#Update room_type column to category data type , same like groupby
room_types["room_type"] = room_types["room_type"].astype("category")

#count variable_room_frequencies 
room_frequencies = room_types["room_type"].value_counts()

print(room_frequencies)

entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64


In [6]:
#Find the timeframe (last review, earliest review) 

#Change data type of last_review column to datetime 
reviews["last_review"] = pd.to_datetime(reviews["last_review"])

#Create first_reviewed, the earliest review date 
first_reviewed = reviews["last_review"].dt.date.min() 

#Create last_review, the latest review date 
last_reviewed = reviews["last_review"].dt.date.max() 

print("Latest review is {}, the earliest review is {}".format(first_reviewed,last_reviewed))

Latest review is 2019-01-01, the earliest review is 2019-07-09


In [8]:
#Joining the Dataframe 
#Merge prices and room_types to create rooms_and_prices
room_and_prices = prices.merge(room_types, how = "outer", on = "listing_id")

#Merge room_and_prices with reviews 
airbnb_merged = room_and_prices.merge(reviews,how="outer", on = "listing_id")

#Drop missing values from Airbnb_merged
airbnb_merged.dropna(inplace=True)

#Check for duplicate value 
print("There are {} duplicates in the DataFrame.".format(airbnb_merged.dropna))

There are <bound method DataFrame.dropna of        listing_id  price                 nbhood_full  price_per_month  \
0            2595  225.0          Manhattan, Midtown      6843.750000   
1            3831   89.0      Brooklyn, Clinton Hill      2707.083333   
2            5099  200.0      Manhattan, Murray Hill      6083.333333   
3            5178   79.0   Manhattan, Hell's Kitchen      2402.916667   
4            5238  150.0        Manhattan, Chinatown      4562.500000   
...           ...    ...                         ...              ...   
25197    36425863  129.0  Manhattan, Upper East Side      3923.750000   
25198    36427429   45.0            Queens, Flushing      1368.750000   
25199    36438336  235.0  Staten Island, Great Kills      7147.916667   
25200    36442252  100.0           Bronx, Mott Haven      3041.666667   
25201    36455809   30.0          Brooklyn, Bushwick       912.500000   

                                      description        room_type  \
0        

In [9]:
#Analyst listing prices by NYC borough 

#Extract information from the nbhood_full column and store as new column, borough 
airbnb_merged["borough"] = airbnb_merged["nbhood_full"].str.partition(",")[0]

#Group by borough and calculate summary statistics
boroughs = airbnb_merged.groupby("borough")["price"].agg(["sum","mean","median","count"])

#Round boroughs to 2 decimal places, and sort by mean in descending order 
boroughs = boroughs.round(2).sort_values("mean",ascending=False)

print(boroughs)

                     sum    mean  median  count
borough                                        
Manhattan      1898417.0  184.04   149.0  10315
Brooklyn       1275250.0  122.02    95.0  10451
Queens          320715.0   92.83    70.0   3455
Staten Island    22974.0   86.04    71.0    267
Bronx            55156.0   79.25    65.0    696


In [10]:
#Create labels for price range, label_names 
label_names = ["Budget","Average","Expensive","Extravagant"]

#Label range 
ranges = [0,69,175,350,np.inf]

#Insert new column, price range, into Dataframe 
airbnb_merged["price_range"]= pd.cut(airbnb_merged["price"], bins = ranges, labels = label_names)

#Calculate borough and price_range frequencies, prices_by_borough
prices_by_borough = airbnb_merged.groupby(["borough","price_range"])["price_range"].count()
print(prices_by_borough)

borough        price_range
Bronx          Budget          381
               Average         285
               Expensive        25
               Extravagant       5
Brooklyn       Budget         3194
               Average        5532
               Expensive      1466
               Extravagant     259
Manhattan      Budget         1148
               Average        5285
               Expensive      3072
               Extravagant     810
Queens         Budget         1631
               Average        1505
               Expensive       291
               Extravagant      28
Staten Island  Budget          124
               Average         123
               Expensive        20
               Extravagant       0
Name: price_range, dtype: int64


In [13]:
airbnb_merged.to_excel(r'C:\Users\cangn\Downloads\project\NYC Airbnb Data Analysis\Airbnb_merged.xlsx',index=False)